<div style="border-radius:10px;
            border : #015a2c solid;
            background-color:#ecfff5;
           font-size:110%;
           letter-spacing:0.5px;
            text-align: center">

<center><h1 style="padding: 25px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
House price prediction ⛪🏤🏡</h1></center>
<center><h3 style="padding-bottom: 25px; color:#015a2c; font-weight: bold; font-style:italic; font-family: Cursive">
(linear and nonlinear models)</h3></center>     

</div>

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

# ----------------------------------------------------
import sklearn
import scipy
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ----------------------------------------------------
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, SelectPercentile

# ----------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso

# ----------------------------------------------------
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ----------------------------------------------------
sns.set()
sns.set_style("white")
import warnings
warnings.filterwarnings("ignore")

# Define Functions

In [ ]:
def outlier_detect_new(df, col):
    q1_col = Q1[col]
    iqr_col = IQR[col]
    q3_col = Q3[col]
    return df[((df[col] < (q1_col - 1.5 * iqr_col)) |(df[col] > (q3_col + 1.5 * iqr_col)))]

# ----------------------------------------------------------
def lower_outlier(df, col):
    q1_col = Q1[col]
    iqr_col = IQR[col]
    q3_col = Q3[col]
    lower = df[(df[col] < (q1_col - 1.5 * iqr_col))]
    return lower

# ----------------------------------------------------------
def upper_outlier(df, col):
    q1_col = Q1[col]
    iqr_col = IQR[col]
    q3_col = Q3[col]
    upper = df[(df[col] > (q3_col + 1.5 * iqr_col))]
    return upper

# ----------------------------------------------------------
def preprocess(df, col):
    print("******************** {} ********************\n".format(col))
    plt.figure(figsize=(16,4))
    plt.subplot(1,2,1)
    df[col].plot(kind='box', subplots=True, sharex=False, vert=False)
    plt.subplot(1,2,2)
    df[col].plot(kind='density', subplots=True, sharex=False)
    plt.show()

# ----------------------------------------------------------
def preprocess_cat(df, col):
    print("******************** {} ********************\n".format(col))
    df[col].value_counts().plot(kind='bar')
    plt.xticks(rotation='vertical')
    plt.show()
    
# ----------------------------------------------------------
def replace_upper(df, col):
    q1_col = Q1[col]
    iqr_col = IQR[col]
    q3_col = Q3[col]
    tmp = 9999999
    upper = q3_col + 1.5 * iqr_col
    df[col] = df[col].where(lambda x: (x < (upper)), tmp)
    df[col] = df[col].replace(tmp, upper)

# ----------------------------------------------------------
def replace_lower(df, col):
    q1_col = Q1[col]
    iqr_col = IQR[col]
    q3_col = Q3[col]
    tmp = 1111111
    lower = q1_col - 1.5 * iqr_col
    df[col] = df[col].where(lambda x: (x > (lower)), tmp)
    df[col] = df[col].replace(tmp, lower)

# ----------------------------------------------------------
def replace_mode(df, col):
    df[col] = df[col].fillna(df[col].mode()[0])

# ----------------------------------------------------------
def build_model(x,y):
    x = sm.add_constant(x)
    lm = sm.OLS(y,x).fit()
    print(lm.summary())
    return x, lm

# ----------------------------------------------------------
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

# Data Understanding

In [ ]:
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print("train: {}".format(train.shape))
print("test: {}".format(test.shape))

In [ ]:
train.describe().T

In [ ]:
print("train: {}".format(train.info()))
print("\n************************************************************\n")
print("test: {}".format(test.info()))

In [ ]:
pd.set_option('display.max_rows', train.shape[0])
pd.set_option('display.max_rows', test.shape[0])
pd.DataFrame(train.isnull().sum().sort_values(ascending = False))

In [ ]:
pd.DataFrame(test.isnull().sum().sort_values(ascending = False))

In [ ]:
train.plot(kind='density', subplots=True, layout=(14,6), sharex=False, figsize= (40,80))
plt.show()

# Preprocessing

In [ ]:
train.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
            'FireplaceQu', 'LotFrontage'], axis=1, inplace=True)

test.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
           'FireplaceQu', 'LotFrontage'], axis=1, inplace=True)

In [ ]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
categorical_feature = train.dtypes==object
final_categorical_feature = train.columns[categorical_feature].tolist()

#----------------------------------------------------
categorical_feature_test = test.dtypes==object
final_categorical_feature_test = test.columns[categorical_feature_test].tolist()

In [ ]:
numeric_feature = train.dtypes!=object
final_numeric_feature = train.columns[numeric_feature].tolist()

#----------------------------------------------------
numeric_feature_test = test.dtypes!=object
final_numeric_feature_test = test.columns[numeric_feature_test].tolist()

<div style="border-radius:10px;
            background-color:#ffffff;
            letter-spacing:0.5px;">

<h3 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
1. Numerical Field</h3>
</div>

In [ ]:
numeric = train[final_numeric_feature]

#-------------------------------------------
numeric_test = test[final_numeric_feature_test]

numeric.head()

In [ ]:
col_names = numeric.columns
col_names_test = numeric_test.columns

In [ ]:
for i in range(len(col_names)):
    preprocess(train, col_names[i])

In [ ]:
print("train")
for i in range(len(col_names)):
    print("{}: {}".format(col_names[i],(outlier_detect_new(numeric,col_names[i]).shape[0])))
    
print("***********************************")
print("test ")
for i in range(len(numeric_test.columns)):
    print("{}: {}".format(col_names_test[i],(outlier_detect_new(numeric_test,col_names_test[i]).shape[0])))

In [ ]:
discrete_features = ['BsmtFinSF2' ,'LowQualFinSF' ,'BsmtHalfBath' ,'BedroomAbvGr' ,'KitchenAbvGr' ,
                   'EnclosedPorch' ,'3SsnPorch' ,'ScreenPorch' ,'PoolArea' ,'MiscVal' ]

discrete = numeric[discrete_features]
#------------------------------------------------------------------------------------------------------------------
discrete_test = numeric_test[discrete_features]

In [ ]:
continuous = numeric[['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
                   'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
                   'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                   'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
                   'TotRmsAbvGrd', 'Fireplaces','GarageYrBlt', 'GarageCars', 
                   'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                   'MoSold', 'YrSold', 'SalePrice']]

continuous_test = numeric_test[['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
                   'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
                   'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                   'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
                   'TotRmsAbvGrd', 'Fireplaces','GarageYrBlt', 'GarageCars', 
                   'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                   'MoSold', 'YrSold']]
continuous.shape

In [ ]:
continuous_col = continuous.columns
for i in range(len(continuous_col)):
    replace_upper(numeric, continuous_col[i])   
    
#------------------------------------------------------
continuous_col_test = continuous_test.columns
for i in range(len(continuous_col_test)):
    replace_upper(numeric_test, continuous_col_test[i])   

In [ ]:
print("train")
for i in range(len(continuous_col)):
    print("{}: {}".format(continuous_col[i],(upper_outlier(numeric,continuous_col[i]).shape[0])))
    
print("***************************************")

print("test")
for i in range(len(continuous_col_test)):
    print("{}: {}".format(continuous_col_test[i],(upper_outlier(numeric_test,continuous_col_test[i]).shape[0])))

In [ ]:
for i in range(len(continuous_col)):
    replace_lower(numeric, continuous_col[i])
    
#--------------------------------------------------
for i in range(len(continuous_col_test)):
    replace_lower(numeric_test, continuous_col_test[i])

In [ ]:
print("train")
for i in range(len(continuous_col)):
    print("{}: {}".format(continuous_col[i],(lower_outlier(numeric,continuous_col[i]).shape[0])))
    
print("***********************************")

print("test")
for i in range(len(continuous_col_test)):
    print("{}: {}".format(continuous_col_test[i],(lower_outlier(numeric_test,continuous_col_test[i]).shape[0])))

In [ ]:
print("train:\n",numeric.isnull().sum())
print("*********************************")
print("test:\n",numeric_test.isnull().sum())

<div style="border-radius:10px;
            background-color:#ffffff;
            letter-spacing:0.5px;">

<h3 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
2. Categorical feilds</h3>
</div>

In [ ]:
categorical = train[final_categorical_feature]

#---------------------------------------------
categorical_test = test[final_categorical_feature_test]
categorical.head()

In [ ]:
print("train: \n",categorical.isnull().sum())
print("***********************************")
print("test: \n",categorical_test.isnull().sum())

In [ ]:
col_names_cat = categorical.columns
#-----------------------------------------------------------
col_names_cat_test = categorical_test.columns

In [ ]:
for i in range(len(col_names_cat)):
    preprocess_cat(categorical, col_names_cat[i])

In [ ]:
for i in range(len(col_names_cat)):
    replace_mode(categorical, col_names_cat[i])
    
for i in range(len(col_names_cat_test)):
    replace_mode(categorical_test, col_names_cat_test[i])

In [ ]:
print("train: \n",categorical.isnull().sum())
print("***********************************")
print("test: \n",categorical_test.isnull().sum())

In [ ]:
le = LabelEncoder() 
categorical = categorical.apply(lambda col: le.fit_transform(col)) 
categorical_test = categorical_test.apply(lambda col: le.fit_transform(col)) 
categorical.head(5)

In [ ]:
train[col_names_cat] = categorical[col_names_cat]
test[col_names_cat_test] = categorical_test[col_names_cat_test]

In [ ]:
train[col_names] = numeric[col_names]
test[col_names_test] = numeric_test[col_names_test]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Duplicate Value
print("train: \n",train.loc[train.duplicated()].shape)
#--------------------------------------------------
print("test: \n",test.loc[test.duplicated()].shape)

# Normalization & Feature Selection

In [ ]:
mask = np.triu(np.ones_like(train.corr()))
fig, ax = plt.subplots(figsize=(150,120),dpi=80, facecolor='w', edgecolor='k')
sns.heatmap(train.corr(), mask= mask, cmap="YlGnBu", vmax=.2, annot = True, center = 0,
            annot_kws={"fontsize":22})

In [ ]:
correlation=(train.corr()['SalePrice'])
del correlation['SalePrice']
correlation.sort_values(ascending=True).plot(kind='barh',color = "darkgreen", 
                                             figsize = (100,90), fontsize=50)
plt.title("correlation", fontsize=50)
plt.axvline(x = 0.6, ymin = 0, ymax = 1,
            linewidth = 5, linestyle ="--",
            color ='#da1934');
plt.axvline(x = -0.6, ymin = 0, ymax = 1,
            linewidth = 5, linestyle ="--",
            color ='#da1934');

Highly correlated independent variables to target are : `ExterQual` , `BsmtQual`,`OverallQual`, `GrLivArea`, `GarageCars`, `GarageArea`, `TotalBsmtSF` and `1stFlrSF`.

In [ ]:
new_train = train[['Id', 'ExterQual', 'BsmtQual', 'OverallQual', 'GrLivArea', 'GarageCars', 
                   'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'SalePrice']]
#------------------------------------------------------------------------------------------------
new_test = test[['Id', 'ExterQual', 'BsmtQual', 'OverallQual', 'GrLivArea', 'GarageCars', 
                   'GarageArea', 'TotalBsmtSF', '1stFlrSF']]

In [ ]:
x = new_train.drop("SalePrice", axis = 1)
y = new_train['SalePrice']

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y, train_size = 0.7, test_size = 0.3, 
                                                  random_state = 100)

In [ ]:
scale_features = MinMaxScaler()
col = ['ExterQual', 'BsmtQual', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 
       'TotalBsmtSF', '1stFlrSF']

x_train[col] = scale_features.fit_transform(x_train[col])
x_val[col] = scale_features.transform(x_val[col])
#-----------------------------------------------------
new_test[col] = scale_features.transform(new_test[col])

In [ ]:
scale_target = MinMaxScaler()
y_train = pd.DataFrame(y_train)
y_train = scale_target.fit_transform(y_train)

y_val = pd.DataFrame(y_val)
y_val = scale_target.transform(y_val)

In [ ]:
x_train_new = x_train[col]
x_val_new = x_val[col]

In [ ]:
x_train_new, lm = build_model(x_train_new,y_train)

In [ ]:
greater_than_5_per = x_train_new.columns[lm.pvalues > 0.05]

i = 1
while(len(greater_than_5_per)!=0):
    print("iteration {}".format(i))
    new_col = x_train_new.columns[lm.pvalues < 0.05]
    print("num of new columns is {}\nnew columns in iteration are\n{}".format(len(new_col),new_col))
    x_train_new = x_train_new[new_col]
    x_train_new, lm = build_model(x_train_new,y_train)
    #----------------------------------------------------------
    if 'const' in x_train_new.columns[lm.pvalues > 0.05]:
        greater_than_5_per = x_train_new.columns[lm.pvalues > 0.05].drop('const')
    i+=1
    print("\n\n******************************************\n")

<div style="font-family: Cursive; font-size:16px; background-color:#ecfff5; padding: 25px 20px">
Values of p-value are below 5% so we keep the attributes.
</div>

In [ ]:
checkVIF(x_train_new)

In [ ]:
x_train_new = x_train_new.drop('const',axis=1)
new_col = x_train_new.columns
x_val_new = x_val[new_col]
x_val_new.columns

# Models - `with tuned hyperparameters`

In [ ]:
DT = DecisionTreeRegressor(criterion='friedman_mse', max_depth=6, min_samples_leaf=5,
                      random_state=0)
DT.fit(x_train_new, y_train)

y_train_pred_DT = DT.predict(x_train_new)
y_val_pred_DT = DT.predict(x_val_new)

r2_score_DT_train = r2_score(y_train, y_train_pred_DT)
r2_score_DT_val = r2_score(y_val, y_val_pred_DT)
mae_DT = mean_absolute_error(y_val, y_val_pred_DT)

print('R2_score (train): ', r2_score_DT_train)
print('R2_score (test): ', r2_score_DT_val)
print("MAE: ", mae_DT)

In [ ]:
LR=LinearRegression()
LR.fit(x_train_new, y_train)

y_train_pred_LR = LR.predict(x_train_new)
y_val_pred_LR = LR.predict(x_val_new)

r2_score_LR_train = r2_score(y_train, y_train_pred_LR)
r2_score_LR_val = r2_score(y_val, y_val_pred_LR)
mae_LR = mean_absolute_error(y_val, y_val_pred_LR)

print('R2_score (train): ', r2_score_LR_train)
print('R2_score (test): ', r2_score_LR_val)
print("MAE: ", mae_LR)

In [ ]:
Ri= Ridge(alpha=1, fit_intercept=False, solver='saga')
Ri.fit(x_train_new, y_train)

y_train_pred_Ri = Ri.predict(x_train_new)
y_val_pred_Ri = Ri.predict(x_val_new)

r2_score_Ri_train = r2_score(y_train, y_train_pred_Ri)
r2_score_Ri_val = r2_score(y_val, y_val_pred_Ri)
mae_Ri = mean_absolute_error(y_val, y_val_pred_Ri)

print('R2_score (train): ', r2_score_Ri_train)
print('R2_score (test): ', r2_score_Ri_val)
print("MAE: ", mae_Ri)

In [ ]:
EN= ElasticNet(alpha=0.001, fit_intercept=False, l1_ratio=0.0)
EN.fit(x_train_new, y_train)

y_train_pred_EN = EN.predict(x_train_new)
y_val_pred_EN = EN.predict(x_val_new)

r2_score_EN_train = r2_score(y_train, y_train_pred_EN)
r2_score_EN_val = r2_score(y_val, y_val_pred_EN)
mae_EN = mean_absolute_error(y_val, y_val_pred_EN)

print('R2_score (train): ', r2_score_EN_train)
print('R2_score (test): ', r2_score_EN_val)
print("MAE: ", mae_EN)

In [ ]:
KNN = KNeighborsRegressor()
KNN.fit(x_train_new, y_train)

y_train_pred_KNN = KNN.predict(x_train_new)
y_val_pred_KNN = KNN.predict(x_val_new)

r2_score_KNN_train = r2_score(y_train, y_train_pred_KNN)
r2_score_KNN_val = r2_score(y_val, y_val_pred_KNN)
mae_KNN = mean_absolute_error(y_val, y_val_pred_KNN)

print('R2_score (train): ', r2_score_KNN_train)
print('R2_score (test): ', r2_score_KNN_val)
print("MAE: ", mae_KNN)

In [ ]:
RF = RandomForestRegressor(criterion='mae', min_samples_leaf=3, min_samples_split=5,
                      n_estimators=200, random_state=0)
RF.fit(x_train_new, y_train)

y_train_pred_RF = RF.predict(x_train_new)
y_val_pred_RF = RF.predict(x_val_new)

r2_score_RF_train = r2_score(y_train, y_train_pred_RF)
r2_score_RF_val = r2_score(y_val, y_val_pred_RF)
mae_RF = mean_absolute_error(y_val, y_val_pred_RF)

print('R2_score (train): ', r2_score_RF_train)
print('R2_score (test): ', r2_score_RF_val)
print("MAE: ", mae_RF)

In [ ]:
SVR_model = SVR(C=0.1, gamma=0.6)
SVR_model.fit(x_train_new, y_train)

y_train_pred_SVR_model = SVR_model.predict(x_train_new)
y_val_pred_SVR_model = SVR_model.predict(x_val_new)

r2_score_SVR_model_train = r2_score(y_train, y_train_pred_SVR_model)
r2_score_SVR_model_val = r2_score(y_val, y_val_pred_SVR_model)
mae_SVR_model = mean_absolute_error(y_val, y_val_pred_SVR_model)

print('R2_score (train): ', r2_score_SVR_model_train)
print('R2_score (test): ', r2_score_SVR_model_val)
print("MAE: ", mae_SVR_model)

In [ ]:
ET = ExtraTreesRegressor(min_samples_leaf=4, min_samples_split=5, n_estimators=50,
                    random_state=0)
ET.fit(x_train_new, y_train)

y_train_pred_ET = ET.predict(x_train_new)
y_val_pred_ET = ET.predict(x_val_new)

r2_score_ET_train = r2_score(y_train, y_train_pred_ET)
r2_score_ET_val = r2_score(y_val, y_val_pred_ET)
mae_ET = mean_absolute_error(y_val, y_val_pred_ET)

print('R2_score (train): ', r2_score_ET_train)
print('R2_score (test): ', r2_score_ET_val)
print("MAE: ", mae_ET)

In [ ]:
GBR = GradientBoostingRegressor(min_samples_leaf=4, min_samples_split=5,
                          n_estimators=50, random_state=0)
GBR.fit(x_train_new, y_train)

y_train_pred_GBR = GBR.predict(x_train_new)
y_val_pred_GBR = GBR.predict(x_val_new)

r2_score_GBR_train = r2_score(y_train, y_train_pred_GBR)
r2_score_GBR_val = r2_score(y_val, y_val_pred_GBR)
mae_GBR = mean_absolute_error(y_val, y_val_pred_GBR)

print('R2_score (train): ', r2_score_GBR_train)
print('R2_score (test): ', r2_score_GBR_val)
print("MAE: ", mae_GBR)

In [ ]:
XGBR = XGBRegressor(n_estimators = 50, max_depth=3, learning_rate_init=0.2)
XGBR.fit(x_train_new, y_train)

y_train_pred_XGBR = XGBR.predict(x_train_new)
y_val_pred_XGBR = XGBR.predict(x_val_new)

r2_score_XGBR_train = r2_score(y_train, y_train_pred_XGBR)
r2_score_XGBR_val = r2_score(y_val, y_val_pred_XGBR)
mae_XGBR = mean_absolute_error(y_val, y_val_pred_XGBR)

print('R2_score (train): ', r2_score_XGBR_train)
print('R2_score (test): ', r2_score_XGBR_val)
print("MAE: ", mae_XGBR)

# Compare Models

In [ ]:
models = [('Decision Tree Regressor', mae_DT, r2_score_DT_train, r2_score_DT_val,''),
          ('Linear Regression', mae_LR, r2_score_LR_train, r2_score_LR_val,'Stable model-Based on R2'),
          ('Ridge', mae_Ri, r2_score_Ri_train, r2_score_Ri_val,'Stable model-Based on R2'),
          ('ElasticNet', mae_EN, r2_score_EN_train, r2_score_EN_val,'Stable model-Based on R2'),
          ('KNeighbors Regressor', mae_KNN, r2_score_KNN_train, r2_score_KNN_val,''),
          ('Random Forest Regressor', mae_RF, r2_score_RF_train, r2_score_RF_val,''),
          ('SVR', mae_SVR_model, r2_score_SVR_model_train, r2_score_SVR_model_val,'Stable model-Based on R2'),
          ('Extra Trees Regressor', mae_ET, r2_score_ET_train, r2_score_ET_val,'The first best-Based on MAE'),
          ('Gradient Boosting Regressor', mae_GBR, r2_score_GBR_train, r2_score_GBR_val,'The second best-Based on MAE'),
          ('XGBRegressor', mae_XGBR, r2_score_XGBR_train, r2_score_XGBR_val,'')]

In [ ]:
compare_models  = pd.DataFrame(data = models, columns=['Model', 'MAE', 'R2_Score(train)', 'R2_Score(test)', 'Description'])
compare_models.style.background_gradient(cmap='YlGn')

In [ ]:
plt.figure(figsize=(10,5))
compare_models.sort_values(by=['MAE'], ascending=True, inplace=True)
sns.barplot(x='MAE', y='Model', data = compare_models, palette='plasma')

# Final Model

In [ ]:
plt.figure(figsize=[15,5])

c = [i for i in range(1,439,1)]
d = [i for i in range(1,439,1)]

plt.plot(c, y_val, color="blue", linewidth=1, linestyle="-")
plt.plot(d, y_val_pred_ET, color="red",  linewidth=1, linestyle="-")
plt.xlabel('Index', fontsize=18) 
plt.ylabel('SalePrice', fontsize=16)

In [ ]:
y_val = y_val.ravel()
actualvspredicted = pd.DataFrame({"Actual":y_val,"Predicted":y_val_pred_ET, 'Different':y_val-y_val_pred_ET})
actualvspredicted.head(10).style.background_gradient(cmap='YlGn')

In [ ]:
plt.figure(figsize=[8,6])
p = sns.jointplot(actualvspredicted['Predicted'],actualvspredicted['Actual'], kind='reg')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Submission

In [ ]:
new_test.columns

In [ ]:
test_final_feature = new_test[new_col]

In [ ]:
predictions = ET.predict(test_final_feature)
predictions = predictions.reshape(-1,1)

inverse_pred = scale_target.inverse_transform(predictions)
inverse_pred = inverse_pred.ravel()

In [ ]:
output = pd.DataFrame({'Id': new_test.Id, 'SalePrice': inverse_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #015a2c;
            letter-spacing:0.5px;">

<center><h4 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
    Thanks for your attention and for reviewing my notebook.🙌 <br><br>Please write your comments for me.📝</h4></center>
<center><h4 style="padding: 5px 0px; color:#015a2c; font-weight: bold; font-family: Cursive">
If you liked my work and found it useful, please upvote. Thank you🙏</h4></center>
</div>